# 패키지 설치

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경 변수를 불러옵니다.

# api_key = os.getenv("OPENAI_API_KEY")
# print(f"API Key: {api_key}")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [2]:
# print('hello world')
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores.faiss import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

prompt = PromptTemplate.from_template("Question: {question}\nAnswer:")

Question = '당신은 누구십니까?'

response = llm.invoke(prompt.format(question=Question))

In [4]:
response

AIMessage(content='저는 인공지능 챗봇입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 25, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4d465ffc-8806-4f67-bedb-2b6fa95c36bc-0', usage_metadata={'input_tokens': 25, 'output_tokens': 15, 'total_tokens': 40})

In [5]:
import requests

urls = {
    "namu_crawler.py":"https://raw.githubusercontent.com/lymanstudio/proj_artist_info_gen/main/namu_crawler.py",
    "namu_loader.py":"https://raw.githubusercontent.com/lymanstudio/proj_artist_info_gen/main/namu_loader.py"
}

for key, val in urls.items():    
    r = requests.get(val)

    # 다운로드한 파일을 현재 디렉토리에 저장
    with open(key, "w", encoding='utf-8') as file:
        file.write(r.text)


In [6]:
# downloaded_module에서 클래스를 임포트
from namu_crawler import NamuCrawler

# MyClass의 인스턴스를 생성하고 메서드를 호출
my_instance = NamuCrawler("https://namu.wiki/w/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8", 3)
my_instance.construct_toc()
my_instance.get_doc_title()

'프로미스나인'

In [7]:
my_instance.get_content_heading('s-6')

(('6.', '활동'),
 '/w/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8/%ED%99%9C%EB%8F%99')

In [ ]:
my_instance.construct_toc()
my_instance.print_toc()

In [93]:
from namu_loader import NamuLoader
my_instance2 = NamuLoader("https://namu.wiki/w/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8", 3)

In [94]:
my_instance2.get_total_content()

TypeError: NamuLoader.get_total_content() missing 3 required positional arguments: 'parent_item', 'sub_url', and 'hop'

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://namu.wiki/w/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8'
hop = 3 # 현재 문서가 몇번째 hop(depth)으로 크롤링 되고 있는지
session = requests.Session()
html_doc = session.get(url)
soup = BeautifulSoup(html_doc.text, 'html.parser')

In [26]:
print(soup.prettify())

<!DOCTYPE html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ko%22%7D%7D" data-n-head-ssr="" lang="ko">
 <head>
  <title>
   프로미스나인 - 나무위키
  </title>
  <meta charset="utf-8" data-n-head="ssr"/>
  <meta content="user-scalable=no, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0, width=device-width" data-n-head="ssr" name="viewport"/>
  <meta content="ie=edge" data-n-head="ssr" http-equv="x-ua-compatible"/>
  <meta content="the seed" data-n-head="ssr" name="generator"/>
  <meta content="yes" data-n-head="ssr" name="mobile-web-app-capable"/>
  <meta content="나무위키" data-n-head="ssr" name="application-name"/>
  <meta content="나무위키" data-n-head="ssr" name="msapplication-tooltip"/>
  <meta content="/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C%80%EB%AC%B8" data-n-head="ssr" name="msapplication-starturl"/>
  <meta content="max-image-preview:large" data-n-head="ssr" name="robots"/>
  <meta content="프로미스나인" data-n-head="ssr" property="og:title"/>
  <meta content="나무위키" data-n-head="s

In [ ]:
html_str = str(soup)
start_pos = html_str.find(str('<body>')) + len(str('<body>'))
toc_str = str(soup.find("div", class_='wiki-macro-toc'))
end_pos = html_str.find(toc_str)
between_content = html_str[start_pos:end_pos]
soup_between = BeautifulSoup(between_content, 'html.parser')

In [ ]:
print(soup_between.prettify())

In [23]:
toc_dict = dict()
if len(profile := soup_between.find_all('div', class_ = 'wiki-table-wrap table-right')) > 0:
    toc_dict['s-p'] = (("0.",'PROFILE'), profile[-1]) # dictionary의 첫번째 아이템으로 넣기

In [27]:
toc = soup.find("div", class_ = 'toc-indent')
pattern = r'^(\d+\.)+' # 제목 파싱용 패턴
for i, e in enumerate(toc.find_all("span", class_  = "toc-item")):
    item_value = e.get_text()
    numbers = re.match(pattern, item_value).group()
    text = re.sub(pattern, '', item_value).strip()
    toc_dict[e.find('a')['href'].replace("#", "")] = ((numbers, text), soup.find('a', id=e.find('a')['href'].replace("#", "")))


In [31]:
"""마지막 원소로 각주영역 저장, 없다면 None 저장"""
toc_dict['s-f'] = (("EOD.", 'FOOTNOTES'), soup.find("div", class_ = 'wiki-macro-footnote')) # 마지막엔 각주 영역


In [71]:
toc_dict['s-1'][1]

<a data-v-c693aea7="" href="#toc" id="s-1">1.</a>

In [51]:
soup.find("a", href = "/member/star/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8").get_text()

' 164'

In [73]:
ele = soup.find_all('div', class_='wiki-paragraph')

In [90]:
for e1 in ele[180]:
    print(e1)

<div data-v-c693aea7="" style="display:inline-block;padding:12px 15px 10px 15px;border:2px dashed #ccc;border-left:5px solid #da75a1;background:rgba(40, 69, 111, .02)"><span class="wiki-size-down-1" data-v-c693aea7="">인사드리겠습니다. 둘, 셋!<br data-v-c693aea7=""/>(<a class="wiki-link-internal" data-v-c693aea7="" href="/w/To.%20Day#CLOVER" title="To. Day">이제는 약속해 소중히 간직해</a>, 당신의 아이돌로 성장하겠습니다!)<a class="wiki-fn-content" data-v-c693aea7="" href="#fn-6" title="My Little Society 활동 이후로 해당 부분은 생략하고 있다. 오랜 팬들은 뒷부분은 빼더라도 앞부분까지는 해주길 바란다는 의견도 있으나, 내부적으로 인사법을 아예 하지 않기로 결정한 듯하다."><span data-v-c693aea7="" id="rfn-6"></span>[6]</a><br data-v-c693aea7=""/>안녕하세요, 프로미스나인입니다!</span></div>
<br data-v-c693aea7=""/>
<a class="wiki-link-internal" data-v-c693aea7="" href="/w/%EC%97%A0%EB%84%B7" title="엠넷">엠넷</a>
의 《
<a class="wiki-link-internal" data-v-c693aea7="" href="/w/%EC%95%84%EC%9D%B4%EB%8F%8C%ED%95%99%EA%B5%90" title="아이돌학교">아이돌학교</a>
》에서 선발되어 
<a class="wiki-link-internal" data-v-c693aea7="" href="/w/2018